In [1]:
import re
import mysql.connector
from bs4 import BeautifulSoup

In [2]:
class Preprocess:
    
    '''data preprocessing'''
    
    def __init__(self,data, table_name, col_names):
        
        self.data = data
        self.table_name = table_name
        self.col_names = col_names
        self.soup = BeautifulSoup(self.data, "lxml")

    def create(self):

        H = []
        temp = []
        for table in self.soup.find_all("table"):
            try:
                for sample in table.tbody.find_all("td"):
                    temp.append(sample.get_text().strip())
                    if '|' in sample.get_text():
                        # se la barretta è stata messa nel punto sbagliato, proviamo comunque a salvare l'ultima colonna
                        if len(re.findall('^[A-Za-z0-9]',sample.get_text().strip()))>0:
                            temp[-1] = re.sub('\|','',temp[-1]).strip()
                            #print(temp)
                            H.append(temp)
                        else:
                            #print(temp[-1])
                            H.append(temp[:-1])
                        temp = []
            except:
                continue
        return H

    def clean(self,H):
        H_new = []
        for h in H:
            #if len(h) == len(H[0]):    #attention!!!!
            for i in range(len(h)):
                h[i] = re.sub('[A-Z0-9a-z]+:','',h[i])
                h[i] = re.sub(r"[^A-Z0-9a-z\'\- \,]", '',h[i])
                h[i] = re.sub('199x|199X|19yy', "3000", h[i])
                h[i] = re.sub('19aa|19xx', "1900", h[i])
                h[i] = re.sub('195x', "1950", h[i])
                h[i] = re.sub('194x', "1940", h[i])
                h[i] = re.sub(r"^Un$", "1", h[i])
            H_new.append(h)
        return H_new
    
    
    def insert_values(self,H):
        ex = 0
        print(self.table_name,self.col_names)
        for i in range(len(H)):

            add_row1 = 'INSERT INTO '+self.table_name+' ('
            add_row2 = ' VALUES ('
            data_row = []
            for j in range(len(H[i])): 
                if H[i][j]!='' and j<len(self.col_names):
                    data_row.append(H[i][j])
                    add_row1 += self.col_names[j]+','
                    add_row2 += '%s'+','
            add_row1 = add_row1[:-1]+')'
            add_row2 = add_row2[:-1]+')'
            add_row = add_row1 + add_row2

            try:
                cursor.execute(add_row, data_row)
                cnx.commit()
            except mysql.connector.ProgrammingError as e:
                print(i,add_row,'\n',data_row,'\n',e)
                ex +=1
            except mysql.connector.DataError as e:
                print(i,data_row,'\n',e)
                ex +=1
            except mysql.connector.DatabaseError as e:
                print(i,H[i],'\n',data_row,'\n',e)
                ex +=1
        print(ex)


    def preprocess(self):

        H = self.create(self.soup)
        if self.table_name == "CASTS":
            H = H[22:]
            H = [[h[0]]+h[2:] for h in H if len(h)>2] #eliminiamo title
            H = self.clean(H)
        elif self.table_name == 'SAYINGS':
            H = [[h[0]]+h[2:] for h in H if len(h)>2] #eliminiamo title
            H = self.clean(H)
        elif self.table_name == 'REMAKES':
            H = [[h[0]]+h[3:5] for h in H if len(h)>4] #eliminiamo title e year (2 volte)
            H = self.clean(H)
        elif self.table_name == 'SYNONYMS':
            H = [h[:-1] for h in H] #eliminiamo title
            H = self.clean(H)
        else:
            H = self.clean(H)
        self.insert_values(H)

        return "Ok"

In [3]:
# FILE
with open("data/Movies_ACTORS.html", 'r') as f:
    actors = f.read()
with open('data/Movies_AWARDS.html', 'r') as f:
    awards = f.read()
with open('data/Movies_CASTS.html', 'r') as f:
    casts = f.read()
with open('data/Movies_MAIN.html', 'r') as f:
    movies = f.read()    
with open('data/Movies_PEOPLE.html', 'r') as f:
    people = f.read()
with open('data/Movies_REMAKES.html', 'r') as f:
    remakes = f.read()
with open('data/Movies_SAYINGS.html', 'r') as f:
    sayings = f.read()
with open('data/Movies_STUDIOS.html', 'r') as f:
    studios = f.read()
with open('data/Synonyms.html', 'r') as f:
    synonyms = f.read()
#with open('data/ColorProcess.html','r') as f:
#    colors = f.read()

database and tables creation

In [4]:
tableNames = ["AWARDS", "MOVIES", "CASTS", "ACTORS", "PEOPLE", "REMAKES", "SAYINGS", "STUDIOS", "SYNONYMS"]

colActors =    ["StageName","DateWork","RealSurname","RealName","Gender","DateBirth","DateDeath","TypeRole","Notes"]
colAwards =    ["AwardsID","AwardingOrg","Country","Colloquial","Year"]
colCasts  =    ['MovieId', 'Actor', 'Type','Role', 'Award']
colMovies  =   ['MovieId', 'Title', 'Year', 'Director', 'Producers', 'Studios', 'ColorProcess', 'Category', 'Awards', 'Location']
colPeople  =   ["RefName","Codes","DirectorId","Years","LastName","FirstName","DateBirth","DateDeath","Background","Notes"]
colRemakes  =  ["MovieId","Fraction","PriorFilm"]
colSayings  =  ["MovieId","Actor","col_As","col_To","Saying"]
colStudios  =  ["Name","FullCompanyName","City","Country","Year","First","Founder","Successor","Notes"]
colSynonyms  = ["MovieId","AlternativeTitle","Secondary","Country"]

files = {"ACTORS":actors,"AWARDS": awards,"CASTS":casts,"MOVIES":movies,"PEOPLE":people, 
        "REMAKES":remakes,"SAYINGS":sayings,"STUDIOS":studios,"SYNONYMS":synonyms}

colNames = {"ACTORS":colActors,"AWARDS":colAwards,"CASTS":colCasts,"MOVIES":colMovies,"PEOPLE":colPeople, 
            "REMAKES":colRemakes,"SAYINGS":colSayings,"STUDIOS":colStudios,"SYNONYMS":colSynonyms}

# Each ID of the dictionary is a name of a table
TABLES = {}

TABLES['ACTORS'] = (
    "CREATE TABLE `ACTORS` ("
    "  `StageName` varchar(128) NOT NULL,"
    "  `DateWork` varchar(128),"
    "  `RealSurname` varchar(128),"
    "  `RealName` varchar(128),"
    "  `Gender` varchar(128),"
    "  `DateBirth` integer(11),"
    "  `DateDeath` integer(11),"
    "  `TypeRole` varchar(128),"
    "  `Notes` varchar(128),"
    "  PRIMARY KEY (`StageName`)"
    ") ENGINE=InnoDB")

TABLES['CASTS'] = (
    "CREATE TABLE `CASTS` ("
    "  `MovieId` varchar(100),"
    "  `Actor` varchar(100),"
    "  `Type` varchar(100),"
    "  `Role` varchar(100),"
    "  `Award` varchar(150),"
    "   FOREIGN KEY (`MovieId`) "
    "   REFERENCES `MOVIES` (`MovieId`),"
#    "   FOREIGN KEY (`Award`) "
#    "   REFERENCES `AWARDS` (`AwardsID`),"
    "  PRIMARY KEY (`MovieId`,`Actor`)"
    ") ENGINE=InnoDB")

TABLES['MOVIES'] = (
    "CREATE TABLE `MOVIES` ("
    "  `MovieId` varchar(128) NOT NULL UNIQUE,"
    "  `Title` varchar(128) NOT NULL,"
    "  `Year` int(11),"
    "  `Director` varchar(128),"
    "  `Producers` varchar(128),"
    "  `Studios` varchar(128),"
    "  `ColorProcess` varchar(128),"
    "  `Category` varchar(128),"
    "  `Awards` varchar(128),"
    "  `Location` varchar(128),"
#    "   FOREIGN KEY (`Awards`) "
#    "   REFERENCES `AWARDS` (`AwardsID`),"
    "  PRIMARY KEY (`MovieId`)"
    ") ENGINE=InnoDB")

TABLES['SAYINGS'] = (
    "CREATE TABLE `SAYINGS` ("
    "  `MovieId` varchar(100) NOT NULL,"
    "  `Actor` varchar(100),"
    "  `col_As` varchar(100),"
    "  `col_To` varchar(100),"
    "  `Saying` varchar(1000) NOT NULL,"
    "   FOREIGN KEY (`MovieId`) "
    "   REFERENCES `MOVIES` (`MovieId`),"
    "  PRIMARY KEY (`Saying`)"
    ") ENGINE=InnoDB")

TABLES['PEOPLE'] = (
    "CREATE TABLE `PEOPLE` ("
    "  `RefName` varchar(100),"
    "  `Codes` varchar(100),"
    "  `DirectorId` varchar(100),"
    "  `Years` varchar(100),"
    "  `LastName` varchar(100),"
    "  `FirstName` varchar(100),"
    "  `DateBirth` integer(100),"
    "  `DateDeath` integer(100),"
    "  `Background` varchar(100), "
    "  `Notes` varchar(100),"
    "  PRIMARY KEY (`RefName`)"
    ") ENGINE=InnoDB")

TABLES['AWARDS'] = (
    "CREATE TABLE `AWARDS` ("
    "  `AwardsID` varchar(128),"
    "  `AwardingOrg` varchar(100),"
    "  `Country` varchar(100),"
    "  `Colloquial` varchar(100),"
    "  `Year` integer(100),"
    "  PRIMARY KEY (`AwardsID`)"
    ") ENGINE=InnoDB")

TABLES['REMAKES'] = (
    "CREATE TABLE `REMAKES` ("
    "  `MovieId` varchar(128),"
    "  `Fraction` varchar(100),"
    "  `PriorFilm` varchar(100),"
    "   FOREIGN KEY (`MovieId`) "
    "   REFERENCES `MOVIES` (`MovieId`),"
    "   FOREIGN KEY (`PriorFilm`) "
    "   REFERENCES `MOVIES` (`MovieId`),"
    "  PRIMARY KEY (`MovieId`,`PriorFilm`)"
    ") ENGINE=InnoDB")

TABLES['STUDIOS'] = (
    "CREATE TABLE `STUDIOS` ("
    "  `Name` varchar(100),"
    "  `FullCompanyName` varchar(100),"
    "  `City` varchar(100),"
    "  `Country` varchar(100),"
    "  `Year` integer(100),"
    "  `First` varchar(100),"
    "  `Founder` varchar(100),"
    "  `Successor` varchar(100),"
    "  `Notes` varchar(100),"
    "  PRIMARY KEY (`Name`)"
    ") ENGINE=InnoDB")

TABLES['SYNONYMS'] = (
    "CREATE TABLE `SYNONYMS` ("
    "  `MovieId` varchar(128),"
    "  `AlternativeTitle` varchar(128),"
    "  `Secondary` varchar(100),"
    "  `Country` varchar(128),"
    "   FOREIGN KEY (`MovieId`) "
    "   REFERENCES `MOVIES` (`MovieId`),"
    "  PRIMARY KEY (`MovieId`)"
    ") ENGINE=InnoDB")


In [5]:
cnx = mysql.connector.connect(user='', password='',host='')
cursor = cnx.cursor()

c = "DROP DATABASE IF EXISTS `DB1`"
cursor.execute(c)
    
c = "CREATE DATABASE `DB1`"
cursor.execute(c)
c = "USE `DB1`"
cursor.execute(c)



for name in tableNames:
    cursor.execute(TABLES[name])